In [1]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR/single-kmer-match

threads=1

These commands test out fishing for a specific AMR gene kmer in reads, assembling them, and then trying to pull out more kmers to extend the assembled contig until we get an AMR gene match.

# Construct k-mer probe

In [68]:
grep 'sul3' kmer-probe/nucleotide_fasta_protein_homolog_model.fasta -A 1 | tee kmer-probe/sul3.fasta
bp_seq_length kmer-probe/sul3.fasta

>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTTTATTTAGATACAGATAAGGCAATTGAGCATGCTCTGCATTTGGTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACTGAAGTGGGCGTTGTGGAAGAAATCAAAAGACTCAAACCTGTCATTAAGGCTTTAAAAGAAAAAGGCATTTCTATTTCTGTTGATACATTTAAACCTGAGGTTCAGAGTTTTTGCATAGAACAAAAGGTTGATTTTATTAATGATATTCAAGGTTTTCCTTATCCTGAGATTTATTCAGGCTTGGCAAAGTCAGATTGCAAACTTGTGTTGATGCACTCCGTTCAGCGAATTGGTGCAGCTACTAAAGTTGAAACGAATCCGGAAGAGGTTTTTACTTCCATGATGGAATTTTTTAAAGAAAGAATTGCTGCTTTAGTTGAGGCTGGTGTAAAGCGTGAACGAATTATTCTTGATCCGGGTATGGGCTTCTTTTTAGGCTCTAATCCAGAAACATCTATTCTTGTTTTGAAGCGTTTCCCTGAAATTCAAGAAGCTTTTAATTTGCAAGTAATGATTGCAGTGTCACGGAAATCATTCTTAGGTAAAATAACTGGAACCGATGTGAAATCTCGTTTAGCACCAACTCTTGCAGCAGAAATGTATGCATACAAAAAAGGTGCAGATTATCTCCGCACCCATGATGTTAAGTCTTTATCAGATGCCTTGAAAATATCCAAAGCCCTAGGTTAG
kmer-probe/sul3.fasta 792 bp 1 sequences


Pick a single kmer from the middle of this gene.

In [4]:
kmer='TGTGTTGATGCACTCCGTTCAGCGAATTGGT'
kmer_length=31
echo -n ${kmer} | wc -c
sed -e "s/\(${kmer}\)/ [\1] /" kmer-probe/sul3.fasta

31
>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTTTATTTAGATACAGATAAGGCAATTGAGCATGCTCTGCATTTGGTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACTGAAGTGGGCGTTGTGGAAGAAATCAAAAGACTCAAACCTGTCATTAAGGCTTTAAAAGAAAAAGGCATTTCTATTTCTGTTGATACATTTAAACCTGAGGTTCAGAGTTTTTGCATAGAACAAAAGGTTGATTTTATTAATGATATTCAAGGTTTTCCTTATCCTGAGATTTATTCAGGCTTGGCAAAGTCAGATTGCAAACT [TGTGTTGATGCACTCCGTTCAGCGAATTGGT] GCAGCTACTAAAGTTGAAACGAATCCGGAAGAGGTTTTTACTTCCATGATGGAATTTTTTAAAGAAAGAATTGCTGCTTTAGTTGAGGCTGGTGTAAAGCGTGAACGAATTATTCTTGATCCGGGTATGGGCTTCTTTTTAGGCTCTAATCCAGAAACATCTATTCTTGTTTTGAAGCGTTTCCCTGAAATTCAAGAAGCTTTTAATTTGCAAGTAATGATTGCAGTGTCACGGAAATCATTCTTAGGTAAAATAACTGGAACCGATGTGAAATCTCGTTTAGCACCAACTCTTGCAGCAGAAATGTATGCATACAAAAAAGGTGCAGATTATCTCCGCACCCATGATGTTAAGTCTTTATCAGATGCCTTGAAAATATCCAAAGCCCTAGGTTAG


In [55]:
(echo -e ">single-kmer\n${kmer}";) | tee kmer-probe/sul3.single.fasta

>single-kmer
TGTGTTGATGCACTCCGTTCAGCGAATTGGT


# Make blast database from this single gene

In [65]:
conda run --name blast makeblastdb -dbtype nucl -in kmer-probe/sul3.fasta



Building a new DB, current time: 01/17/2020 11:14:27
New DB name:   /home/CSCScience.ca/apetkau/workspace/kmer-analysis-test/single-kmer-match/kmer-probe/sul3.fasta
New DB title:  kmer-probe/sul3.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000257969 seconds.


# Iteration 1

## 1.1 Pull out reads matching kmer(s)

In [56]:
conda run --name kat /usr/bin/time -v kat filter seq -t ${threads} -o SRR1952908.kat.1 -m ${kmer_length} -T 0.0000001 \
    --seq ../input/SRR1952908_1.fastq --seq2 ../input/SRR1952908_2.fastq \
    kmer-probe/sul3.single.fasta

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in filter sequence mode
-----------------------------------

Input 1216 is a sequence file.  Counting kmers for input 1216 (kmer-probe/sul3.single.fasta) ... done.  Time taken: 0.1s

Filtering sequences ...
Processed 100000 pairs
Processed 200000 pairs
Processed 300000 pairs
Processed 400000 pairs
Processed 500000 pairs
Processed 600000 pairs
Processed 700000 pairs
Processed 800000 pairs
Processed 900000 pairs
Processed 1000000 pairs
Processed 1100000 pairs
Processed 1200000 pairs
Processed 1300000 pairs
Processed 1400000 pairs
Processed 1500000 pairs
Processed 1600000 pairs
Processed 1700000 pairs
Processed 1800000 pairs
Processed 1900000 pairs
Processed 2000000 pairs
Processed 2100000 pairs
Finished filtering.  Time taken: 108.6s

Found 77 / 2147046 to keep

KAT filter seq completed.
Total runtime: 108.7s

	Command being timed: "kat filter seq -t 1 -o SRR1952908.kat.1 -m 31 -T 0.0000001 --seq ../input/SRR1952908_1.fastq --seq2 ../input/

In [60]:
du -ch *kat.1.*.fastq

20K	SRR1952908.kat.1.in.R1.fastq
20K	SRR1952908.kat.1.in.R2.fastq
40K	total


## 1.2 Assemble reads

In [61]:
conda run --name skesa /usr/bin/time -v skesa --cores 1 \
    --fastq SRR1952908.kat.1.in.R1.fastq,SRR1952908.kat.1.in.R2.fastq \
    --contigs_out SRR1952908.kat.1.contigs.fasta --vector_percent 1

skesa --cores 1 --fastq SRR1952908.kat.1.in.R1.fastq,SRR1952908.kat.1.in.R2.fastq --contigs_out SRR1952908.kat.1.contigs.fasta --vector_percent 1 

Total mates: 154 Paired reads: 77
Reads acquired in  0.000441s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Adapters clip is disabled

Kmer len: 21
Raw kmers: 12239 Memory needed (GB): 0.000234988 Memory available (GB): 30 1 cycle(s) will be performed
Distinct kmers: 689
Kmer count in  0.006960s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Uniq kmers merging in  0.000921s wall, 0.010000s user + 0.000000s system = 0.010000s CPU (n/a%)
Kmers branching in  0.001992s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)

Average read length: 99
Genome size estimate: 289

Kmer: 21 Graph size: 689 Contigs in: 0
Valley: 0

Mark used kmers in  0.000004s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Kmers in multiple/single contigs: 0 0
Fragments before: 1 667
Fragments after: 1 667
New s

## 1.3 Examine assembled sequences

In [67]:
cat SRR1952908.kat.1.contigs.fasta
bp_seq_length SRR1952908.kat.1.contigs.fasta

>Contig_1_19.1064
GGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGCACCAATTCGCTGAACGGAGTGCATCAACACAAGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTCCCAAATCAATCACATCTGCTCCATCTTCAAC
SRR1952908.kat.1.contigs.fasta 631 bp 1 sequences


In [69]:
conda run --name blast /usr/bin/time -v blastn -db kmer-probe/sul3.fasta -query SRR1952908.kat.1.contigs.fasta

BLASTN 2.9.0+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: kmer-probe/sul3.fasta
           1 sequences; 792 total letters



Query= Contig_1_19.1064

Length=631
                                                                      Score        E
Sequences producing significant alignments:                          (Bits)     Value

gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli]           1166       0.0  


>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
Length=792

 Score = 1166 bits (631),  Expect = 0.0
 Identities = 631/631 (100%), Gaps = 0/631 (0%)
 Strand=Plus/Minus

Query  1    GGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAAC  60
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Sbjct  733  GGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAAC  674

Query  61   GAGATTTCACATCGGTTCCAGT

We matched 631 bp at 100%, but only matched 631/792 bp. Let's try another iteration.

# Iteration 2

## 2.0 Get kmers from ends of contigs

In [70]:
cat SRR1952908.kat.1.contigs.fasta

>Contig_1_19.1064
GGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGCACCAATTCGCTGAACGGAGTGCATCAACACAAGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTCCCAAATCAATCACATCTGCTCCATCTTCAAC


In [10]:
kmer_orig=${kmer}
echo ${kmer_orig}
echo -n ${kmer_orig} | wc -c
kmer1='GGAGATAATCTGCACCTTTTTTGTATGCATA'
echo -n ${kmer1} | wc -c
kmer2='CCAAATCAATCACATCTGCTCCATCTTCAAC'
echo -n ${kmer2} | wc -c
sed -re "s/(${kmer_orig}|${kmer1}|${kmer2})/ [\1] /g" SRR1952908.kat.1.contigs.fasta
(echo -e ">kmer_orig\n${kmer_orig}\n>kmer1\n${kmer1}\n>kmer2\n${kmer2}";) | tee kmer-probe/sul3.2.kmer.fasta

TGTGTTGATGCACTCCGTTCAGCGAATTGGT
31
31
31
>Contig_1_19.1064
 [GGAGATAATCTGCACCTTTTTTGTATGCATA] CATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGCACCAATTCGCTGAACGGAGTGCATCAACACAAGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTC [CCAAATCAATCACATCTGCTCCATCTTCAAC] 
>kmer_orig
TGTGTTGATGCACTCCGTTCAGCGAATTGGT
>kmer1
GGAGATAATCTGCACCTTTTTTGTATGCATA
>kmer2
CCAAATCAATCACATCTGCTCCATCTTCAAC


kmer_orig isn't here in the middle (likely because we need to search for the reverse complement). Let's do this now.

In [11]:
kmer_orig_rc='ACCAATTCGCTGAACGGAGTGCATCAACACA'
sed -re "s/(${kmer_orig_rc})/ [\1] /g" SRR1952908.kat.1.contigs.fasta

>Contig_1_19.1064
GGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGC [ACCAATTCGCTGAACGGAGTGCATCAACACA] AGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTCCCAAATCAATCACATCTGCTCCATCTTCAAC


There it is. So now let's pull out reads matching these 3 kmers (and their reverse complements).

## 2.1 Pull out reads matching kmer(s)

In [82]:
conda run --name kat /usr/bin/time -v kat filter seq -t ${threads} -o SRR1952908.kat.2 -m ${kmer_length} -T 0.0000001 \
    --seq ../input/SRR1952908_1.fastq --seq2 ../input/SRR1952908_2.fastq \
    kmer-probe/sul3.2.kmer.fasta

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in filter sequence mode
-----------------------------------

Input 38080 is a sequence file.  Counting kmers for input 38080 (kmer-probe/sul3.2.kmer.fasta) ... done.  Time taken: 0.1s

Filtering sequences ...
Processed 100000 pairs
Processed 200000 pairs
Processed 300000 pairs
Processed 400000 pairs
Processed 500000 pairs
Processed 600000 pairs
Processed 700000 pairs
Processed 800000 pairs
Processed 900000 pairs
Processed 1000000 pairs
Processed 1100000 pairs
Processed 1200000 pairs
Processed 1300000 pairs
Processed 1400000 pairs
Processed 1500000 pairs
Processed 1600000 pairs
Processed 1700000 pairs
Processed 1800000 pairs
Processed 1900000 pairs
Processed 2000000 pairs
Processed 2100000 pairs
Finished filtering.  Time taken: 109.5s

Found 286 / 2147046 to keep

KAT filter seq completed.
Total runtime: 109.6s

	Command being timed: "kat filter seq -t 1 -o SRR1952908.kat.2 -m 31 -T 0.0000001 --seq ../input/SRR1952908_1.fastq --seq2 ../inp

# 2.2 Assemble reads

In [86]:
conda run --name skesa /usr/bin/time -v skesa --cores 1 \
    --fastq SRR1952908.kat.2.in.R1.fastq,SRR1952908.kat.2.in.R2.fastq \
    --contigs_out SRR1952908.kat.2.contigs.fasta --vector_percent 1

skesa --cores 1 --fastq SRR1952908.kat.2.in.R1.fastq,SRR1952908.kat.2.in.R2.fastq --contigs_out SRR1952908.kat.2.contigs.fasta --vector_percent 1 

Total mates: 572 Paired reads: 286
Reads acquired in  0.003861s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Adapters clip is disabled

Kmer len: 21
Raw kmers: 45509 Memory needed (GB): 0.000873772 Memory available (GB): 30 1 cycle(s) will be performed
Distinct kmers: 1427
Kmer count in  0.023129s wall, 0.020000s user + 0.000000s system = 0.020000s CPU (86.5%)
Uniq kmers merging in  0.001074s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Kmers branching in  0.003712s wall, 0.010000s user + 0.000000s system = 0.010000s CPU (269.4%)

Average read length: 100
Genome size estimate: 170

Kmer: 21 Graph size: 1213 Contigs in: 0
Valley: 0

Mark used kmers in  0.000007s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Kmers in multiple/single contigs: 0 0
Fragments before: 2 1145
Fragments after: 2 1

## 2.3 Examine assembled sequences

In [87]:
cat SRR1952908.kat.2.contigs.fasta
bp_seq_length SRR1952908.kat.2.contigs.fasta

>Contig_1_41.9267
AGTTTAAGTCCTCTTGATTATAGTGTAGGTATTGGAACAGGTTTAGAATTAGGACATTTCCAGTTTATGTTTGGCTATGATTACCCTCTTAGTAACAGTTCGAAAGATCCAGACCTAAGTCAAGTACGCCAACACAACTTCAGAGCAAGTATAGCATATTCATTTAGAAAGCTAAAAAAATAAAACTAACCTAGGGCTTTGGATATTTTCAAGGCATCTGATAAAGACTTAACATCATGGGTGCGGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGCACCAATTCGCTGAACGGAGTGCATCAACACAAGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTCCCAAATCAATCACATCTGCTCCATCTTCAACCAAATGCAGAGCATGCTCAATTGCCTTATCTGTATCTAAATAAAGTCCTCCATCGGAAAAACTATCGGTGGTTATATTTACGATTCCAAAAATCTTGCTCATATCAT

In [88]:
conda run --name blast /usr/bin/time -v blastn -db kmer-probe/sul3.fasta -query SRR1952908.kat.2.contigs.fasta

BLASTN 2.9.0+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: kmer-probe/sul3.fasta
           1 sequences; 792 total letters



Query= Contig_1_41.9267

Length=1057
                                                                      Score        E
Sequences producing significant alignments:                          (Bits)     Value

gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli]           1463       0.0  


>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
Length=792

 Score = 1463 bits (792),  Expect = 0.0
 Identities = 792/792 (100%), Gaps = 0/792 (0%)
 Strand=Plus/Minus

Query  186  CTAACCTAGGGCTTTGGATATTTTCAAGGCATCTGATAAAGACTTAACATCATGGGTGCG  245
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Sbjct  792  CTAACCTAGGGCTTTGGATATTTTCAAGGCATCTGATAAAGACTTAACATCATGGGTGCG  733

Query  246  GAGATAATCTGCACCTTTTT

Awesome. We matched 792/792 bp at 100% identity.

Let's continue on extending.

# Iteration 3

## 3.0: Get kmers from ends

In [12]:
cat SRR1952908.kat.2.contigs.fasta
bp_seq_length SRR1952908.kat.2.contigs.fasta

>Contig_1_41.9267
AGTTTAAGTCCTCTTGATTATAGTGTAGGTATTGGAACAGGTTTAGAATTAGGACATTTCCAGTTTATGTTTGGCTATGATTACCCTCTTAGTAACAGTTCGAAAGATCCAGACCTAAGTCAAGTACGCCAACACAACTTCAGAGCAAGTATAGCATATTCATTTAGAAAGCTAAAAAAATAAAACTAACCTAGGGCTTTGGATATTTTCAAGGCATCTGATAAAGACTTAACATCATGGGTGCGGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGCACCAATTCGCTGAACGGAGTGCATCAACACAAGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTCCCAAATCAATCACATCTGCTCCATCTTCAACCAAATGCAGAGCATGCTCAATTGCCTTATCTGTATCTAAATAAAGTCCTCCATCGGAAAAACTATCGGTGGTTATATTTACGATTCCAAAAATCTTGCTCATATCAT

In [15]:
kmer_orig=${kmer}
kmer_orig_rc='ACCAATTCGCTGAACGGAGTGCATCAACACA'
echo ${kmer_orig}
echo -n ${kmer_orig} | wc -c
kmer1='GGAGATAATCTGCACCTTTTTTGTATGCATA'
echo -n ${kmer1} | wc -c
kmer2='CCAAATCAATCACATCTGCTCCATCTTCAAC'
echo -n ${kmer2} | wc -c
kmer3='AGTTTAAGTCCTCTTGATTATAGTGTAGGTA'
echo -n ${kmer3} | wc -c
kmer4='ATCACTTACGCAAATTGACACATTTTCCCTA'
echo -n ${kmer4} | wc -c
sed -re "s/(${kmer_orig_rc}|${kmer1}|${kmer2}|${kmer3}|${kmer4})/ [\1] /g" SRR1952908.kat.2.contigs.fasta
(echo -e ">kmer_orig\n${kmer_orig}\n>kmer1\n${kmer1}\n>kmer2\n${kmer2}\n>kmer3\n${kmer3}\n>kmer4\n${kmer4}";) \
    | tee kmer-probe/sul3.3.kmer.fasta

TGTGTTGATGCACTCCGTTCAGCGAATTGGT
31
31
31
31
31
>Contig_1_41.9267
 [AGTTTAAGTCCTCTTGATTATAGTGTAGGTA] TTGGAACAGGTTTAGAATTAGGACATTTCCAGTTTATGTTTGGCTATGATTACCCTCTTAGTAACAGTTCGAAAGATCCAGACCTAAGTCAAGTACGCCAACACAACTTCAGAGCAAGTATAGCATATTCATTTAGAAAGCTAAAAAAATAAAACTAACCTAGGGCTTTGGATATTTTCAAGGCATCTGATAAAGACTTAACATCATGGGTGC [GGAGATAATCTGCACCTTTTTTGTATGCATA] CATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGC [ACCAATTCGCTGAACGGAGTGCATCAACACA] AGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGAAATAGAAATGCCTTTTTCTTTTAAAGCCTTAATGACAGGTTTGAGTCTTTTGATTTCTTCCACAACGCCCACTTCAGTTGTATCAGGATTACTGGAAGCGGCTC [CCAAATCAATCACATCTGCTCCATCTTCAAC] CAAATGCAGAGCATGCTCAATTGCCTTATCTGTATCTAAATAAA

## 3.1 Pull out reads matching kmer(s)

In [16]:
conda run --name kat /usr/bin/time -v kat filter seq -t ${threads} -o SRR1952908.kat.3 -m ${kmer_length} -T 0.0000001 \
    --seq ../input/SRR1952908_1.fastq --seq2 ../input/SRR1952908_2.fastq \
    kmer-probe/sul3.3.kmer.fasta

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in filter sequence mode
-----------------------------------

Input 38080 is a sequence file.  Counting kmers for input 38080 (kmer-probe/sul3.3.kmer.fasta) ... done.  Time taken: 0.1s

Filtering sequences ...
Processed 100000 pairs
Processed 200000 pairs
Processed 300000 pairs
Processed 400000 pairs
Processed 500000 pairs
Processed 600000 pairs
Processed 700000 pairs
Processed 800000 pairs
Processed 900000 pairs
Processed 1000000 pairs
Processed 1100000 pairs
Processed 1200000 pairs
Processed 1300000 pairs
Processed 1400000 pairs
Processed 1500000 pairs
Processed 1600000 pairs
Processed 1700000 pairs
Processed 1800000 pairs
Processed 1900000 pairs
Processed 2000000 pairs
Processed 2100000 pairs
Finished filtering.  Time taken: 108.1s

Found 475 / 2147046 to keep

KAT filter seq completed.
Total runtime: 108.3s

	Command being timed: "kat filter seq -t 1 -o SRR1952908.kat.3 -m 31 -T 0.0000001 --seq ../input/SRR1952908_1.fastq --seq2 ../inp

## 3.2 Assemble reads

In [17]:
conda run --name skesa /usr/bin/time -v skesa --cores 1 \
    --fastq SRR1952908.kat.3.in.R1.fastq,SRR1952908.kat.3.in.R2.fastq \
    --contigs_out SRR1952908.kat.3.contigs.fasta --vector_percent 1

skesa --cores 1 --fastq SRR1952908.kat.3.in.R1.fastq,SRR1952908.kat.3.in.R2.fastq --contigs_out SRR1952908.kat.3.contigs.fasta --vector_percent 1 

Total mates: 950 Paired reads: 475
Reads acquired in  0.006307s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Adapters clip is disabled

Kmer len: 21
Raw kmers: 75364 Memory needed (GB): 0.00144699 Memory available (GB): 29.9999 1 cycle(s) will be performed
Distinct kmers: 2008
Kmer count in  0.037942s wall, 0.030000s user + 0.010000s system = 0.040000s CPU (105.4%)
Uniq kmers merging in  0.001272s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Kmers branching in  0.004549s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)

Average read length: 99
Genome size estimate: 120

Kmer: 21 Graph size: 1515 Contigs in: 0
Valley: 0

Mark used kmers in  0.000007s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Kmers in multiple/single contigs: 0 0
Fragments before: 3 1493
Fragments after: 3

## 3.3 Examine assembled contig

In [19]:
cat SRR1952908.kat.3.contigs.fasta
bp_seq_length SRR1952908.kat.3.contigs.fasta

>Contig_1_52.4456
CCCTGCGGGTACTTACTCCTTTGACATACATTTCGGCGATGGCGAGCTTGAGAGCACGCTCACTGCGCATGCCTTTCTCGATGCAGCTGGGATAAAACTCCATCCCGCGAACTTGTGGTACTTTCAAGAGGACTTTCCCGGTCGCAAGATTGAATATTGGATTATTTGGATTTACCAACAAACACTACTATTTAGCTACGAAAACTCAAGAGAAGTACACAAAACCTTTTGGAAGTGATGGAAGTTTAAGTCCTCTTGATTATAGTGTAGGTATTGGAACAGGTTTAGAATTAGGACATTTCCAGTTTATGTTTGGCTATGATTACCCTCTTAGTAACAGTTCGAAAGATCCAGACCTAAGTCAAGTACGCCAACACAACTTCAGAGCAAGTATAGCATATTCATTTAGAAAGCTAAAAAAATAAAACTAACCTAGGGCTTTGGATATTTTCAAGGCATCTGATAAAGACTTAACATCATGGGTGCGGAGATAATCTGCACCTTTTTTGTATGCATACATTTCTGCTGCAAGAGTTGGTGCTAAACGAGATTTCACATCGGTTCCAGTTATTTTACCTAAGAATGATTTCCGTGACACTGCAATCATTACTTGCAAATTAAAAGCTTCTTGAATTTCAGGGAAACGCTTCAAAACAAGAATAGATGTTTCTGGATTAGAGCCTAAAAAGAAGCCCATACCCGGATCAAGAATAATTCGTTCACGCTTTACACCAGCCTCAACTAAAGCAGCAATTCTTTCTTTAAAAAATTCCATCATGGAAGTAAAAACCTCTTCCGGATTCGTTTCAACTTTAGTAGCTGCACCAATTCGCTGAACGGAGTGCATCAACACAAGTTTGCAATCTGACTTTGCCAAGCCTGAATAAATCTCAGGATAAGGAAAACCTTGAATATCATTAATAAAATCAACCTTTTGTTCTATGCAAAAACTCTGAACCTCAGGTTTAAATGTATCAACAGA

# Align to original gene to identify flanking regions

## Align iteration 1

First let's reverse complement the fasta file (so I can easily copy/past alignment in correct orientation).

In [49]:
revseq -sequence SRR1952908.kat.1.contigs.fasta -outseq SRR1952908.kat.1.contigs.rc.fasta
header=`head -n 1 SRR1952908.kat.1.contigs.rc.fasta`
seq_one_line=`tail -n +2 SRR1952908.kat.1.contigs.rc.fasta | tr -d '\n'`
echo -e ">$header\n${seq_one_line}" | tee SRR1952908.kat.1.contigs.rc.single.fasta
bp_seq_length SRR1952908.kat.1.contigs.rc.single.fasta

Reverse and complement a nucleotide sequence
>>Contig_1_19.1064 Reversed:
GTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACTGAAGTGGGCGTTGTGGAAGAAATCAAAAGACTCAAACCTGTCATTAAGGCTTTAAAAGAAAAAGGCATTTCTATTTCTGTTGATACATTTAAACCTGAGGTTCAGAGTTTTTGCATAGAACAAAAGGTTGATTTTATTAATGATATTCAAGGTTTTCCTTATCCTGAGATTTATTCAGGCTTGGCAAAGTCAGATTGCAAACTTGTGTTGATGCACTCCGTTCAGCGAATTGGTGCAGCTACTAAAGTTGAAACGAATCCGGAAGAGGTTTTTACTTCCATGATGGAATTTTTTAAAGAAAGAATTGCTGCTTTAGTTGAGGCTGGTGTAAAGCGTGAACGAATTATTCTTGATCCGGGTATGGGCTTCTTTTTAGGCTCTAATCCAGAAACATCTATTCTTGTTTTGAAGCGTTTCCCTGAAATTCAAGAAGCTTTTAATTTGCAAGTAATGATTGCAGTGTCACGGAAATCATTCTTAGGTAAAATAACTGGAACCGATGTGAAATCTCGTTTAGCACCAACTCTTGCAGCAGAAATGTATGCATACAAAAAAGGTGCAGATTATCTCC
SRR1952908.kat.1.contigs.rc.single.fasta 631 bp 1 sequences


Blasting.

In [42]:
conda run --name blast /usr/bin/time -v blastn -db kmer-probe/sul3.fasta -query SRR1952908.kat.1.contigs.rc.single.fasta

BLASTN 2.9.0+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: kmer-probe/sul3.fasta
           1 sequences; 792 total letters



Query= >Contig_1_19.1064 Reversed:

Length=631
                                                                      Score        E
Sequences producing significant alignments:                          (Bits)     Value

gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli]           1166       0.0  


>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
Length=792

 Score = 1166 bits (631),  Expect = 0.0
 Identities = 631/631 (100%), Gaps = 0/631 (0%)
 Strand=Plus/Plus

Query  1    GTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACT  60
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Sbjct  103  GTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACT  162

Query  61   GAAGTGGGCGTT

## Align to iteration 2

Reverse complement.

In [48]:
revseq -sequence SRR1952908.kat.2.contigs.fasta -outseq SRR1952908.kat.2.contigs.rc.fasta
header=`head -n 1 SRR1952908.kat.2.contigs.rc.fasta`
seq_one_line=`tail -n +2 SRR1952908.kat.2.contigs.rc.fasta | tr -d '\n'`
echo -e ">$header\n${seq_one_line}" | tee SRR1952908.kat.2.contigs.rc.single.fasta
bp_seq_length SRR1952908.kat.2.contigs.rc.single.fasta

Reverse and complement a nucleotide sequence
>>Contig_1_41.9267 Reversed:
TAGGGAAAATGTGTCAATTTGCGTAAGTGATAATTAGTCAATTAAGATAAATGCTACTTTTGTAATCAAAGCAAAATGATATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTTTATTTAGATACAGATAAGGCAATTGAGCATGCTCTGCATTTGGTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACTGAAGTGGGCGTTGTGGAAGAAATCAAAAGACTCAAACCTGTCATTAAGGCTTTAAAAGAAAAAGGCATTTCTATTTCTGTTGATACATTTAAACCTGAGGTTCAGAGTTTTTGCATAGAACAAAAGGTTGATTTTATTAATGATATTCAAGGTTTTCCTTATCCTGAGATTTATTCAGGCTTGGCAAAGTCAGATTGCAAACTTGTGTTGATGCACTCCGTTCAGCGAATTGGTGCAGCTACTAAAGTTGAAACGAATCCGGAAGAGGTTTTTACTTCCATGATGGAATTTTTTAAAGAAAGAATTGCTGCTTTAGTTGAGGCTGGTGTAAAGCGTGAACGAATTATTCTTGATCCGGGTATGGGCTTCTTTTTAGGCTCTAATCCAGAAACATCTATTCTTGTTTTGAAGCGTTTCCCTGAAATTCAAGAAGCTTTTAATTTGCAAGTAATGATTGCAGTGTCACGGAAATCATTCTTAGGTAAAATAACTGGAACCGATGTGAAATCTCGTTTAGCACCAACTCTTGCAGCAGAAATGTATGCATACAAAAAAGGTGCAGATTATCTCCGCACCCATGATGTTAAGTCTTTATCAGATGCCTTGAAAATATCCAAAGCCCTAGGTTAGTTTTATTTTTTTAGCTTTCTAAATGAATATGCTATACTTGCTCTGAAGTTGTGT

Now let's do a BLAST alignment.

In [36]:
conda run --name blast /usr/bin/time -v blastn -db kmer-probe/sul3.fasta -query SRR1952908.kat.2.contigs.rc.single.fasta

BLASTN 2.9.0+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: kmer-probe/sul3.fasta
           1 sequences; 792 total letters



Query= >Contig_1_41.9267 Reversed:

Length=1057
                                                                      Score        E
Sequences producing significant alignments:                          (Bits)     Value

gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli]           1463       0.0  


>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
Length=792

 Score = 1463 bits (792),  Expect = 0.0
 Identities = 792/792 (100%), Gaps = 0/792 (0%)
 Strand=Plus/Plus

Query  81   ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTT  140
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Sbjct  1    ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTT  60

Query  141  TATTTAGATAC

Let's see where this is in the query (assembled) contig.

In [38]:
query_start='ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTT'
query_end='GCACCCATGATGTTAAGTCTTTATCAGATGCCTTGAAAATATCCAAAGCCCTAGGTTAG'
sed -re "s/(${query_start}|${query_end})/ [\1] /g" SRR1952908.kat.2.contigs.rc.single.fasta

>>Contig_1_41.9267 Reversed:
TAGGGAAAATGTGTCAATTTGCGTAAGTGATAATTAGTCAATTAAGATAAATGCTACTTTTGTAATCAAAGCAAAATGAT [ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTT] TATTTAGATACAGATAAGGCAATTGAGCATGCTCTGCATTTGGTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACTGAAGTGGGCGTTGTGGAAGAAATCAAAAGACTCAAACCTGTCATTAAGGCTTTAAAAGAAAAAGGCATTTCTATTTCTGTTGATACATTTAAACCTGAGGTTCAGAGTTTTTGCATAGAACAAAAGGTTGATTTTATTAATGATATTCAAGGTTTTCCTTATCCTGAGATTTATTCAGGCTTGGCAAAGTCAGATTGCAAACTTGTGTTGATGCACTCCGTTCAGCGAATTGGTGCAGCTACTAAAGTTGAAACGAATCCGGAAGAGGTTTTTACTTCCATGATGGAATTTTTTAAAGAAAGAATTGCTGCTTTAGTTGAGGCTGGTGTAAAGCGTGAACGAATTATTCTTGATCCGGGTATGGGCTTCTTTTTAGGCTCTAATCCAGAAACATCTATTCTTGTTTTGAAGCGTTTCCCTGAAATTCAAGAAGCTTTTAATTTGCAAGTAATGATTGCAGTGTCACGGAAATCATTCTTAGGTAAAATAACTGGAACCGATGTGAAATCTCGTTTAGCACCAACTCTTGCAGCAGAAATGTATGCATACAAAAAAGGTGCAGATTATCTCC [GCACCCATGATGTTAAGTCTTTATCAGATGCCTTGAAAATATCCAAAGCCCTAGGTTAG] TTTTATTTTTTTAGCTTTCTAAATGAATATGCTATACTTGCTCTGAAGTTGTGTTGGCGTACTTGACTTAGGTCTGGATCTTTCGAACTGT

Let's count the flanking sequence bp.

In [40]:
echo -n 'TAGGGAAAATGTGTCAATTTGCGTAAGTGATAATTAGTCAATTAAGATAAATGCTACTTTTGTAATCAAAGCAAAATGAT' | wc -c
echo -n 'TTTTATTTTTTTAGCTTTCTAAATGAATATGCTATACTTGCTCTGAAGTTGTGTTGGCGTACTTGACTTAGGTCTGGATCTTTCGAACTGTTACTAAGAGGGTAATCATAGCCAAACATAAACTGGAAATGTCCTAATTCTAAACCTGTTCCAATACCTACACTATAATCAAGAGGACTTAAACT' | wc -c

80
185


## Align to iteration 3

Reverse complement.

In [47]:
revseq -sequence SRR1952908.kat.3.contigs.fasta -outseq SRR1952908.kat.3.contigs.rc.fasta
header=`head -n 1 SRR1952908.kat.3.contigs.rc.fasta`
seq_one_line=`tail -n +2 SRR1952908.kat.3.contigs.rc.fasta | tr -d '\n'`
echo -e ">$header\n${seq_one_line}" | tee SRR1952908.kat.3.contigs.rc.single.fasta
bp_seq_length SRR1952908.kat.3.contigs.rc.single.fasta

Reverse and complement a nucleotide sequence
>>Contig_1_52.4456 Reversed:
TCAGTTTTCATAATGAATTAAAACAAAAAAATTAGGCTGCAAAGATAGGGAAAATGTGTCAATTTGCGTAAGTGATAATTAGTCAATTAAGATAAATGCTACTTTTGTAATCAAAGCAAAATGATATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTTTATTTAGATACAGATAAGGCAATTGAGCATGCTCTGCATTTGGTTGAAGATGGAGCAGATGTGATTGATTTGGGAGCCGCTTCCAGTAATCCTGATACAACTGAAGTGGGCGTTGTGGAAGAAATCAAAAGACTCAAACCTGTCATTAAGGCTTTAAAAGAAAAAGGCATTTCTATTTCTGTTGATACATTTAAACCTGAGGTTCAGAGTTTTTGCATAGAACAAAAGGTTGATTTTATTAATGATATTCAAGGTTTTCCTTATCCTGAGATTTATTCAGGCTTGGCAAAGTCAGATTGCAAACTTGTGTTGATGCACTCCGTTCAGCGAATTGGTGCAGCTACTAAAGTTGAAACGAATCCGGAAGAGGTTTTTACTTCCATGATGGAATTTTTTAAAGAAAGAATTGCTGCTTTAGTTGAGGCTGGTGTAAAGCGTGAACGAATTATTCTTGATCCGGGTATGGGCTTCTTTTTAGGCTCTAATCCAGAAACATCTATTCTTGTTTTGAAGCGTTTCCCTGAAATTCAAGAAGCTTTTAATTTGCAAGTAATGATTGCAGTGTCACGGAAATCATTCTTAGGTAAAATAACTGGAACCGATGTGAAATCTCGTTTAGCACCAACTCTTGCAGCAGAAATGTATGCATACAAAAAAGGTGCAGATTATCTCCGCACCCATGATGTTAAGTCTTTATCAGATGCCTTGAAAATATCCAAAGCCCTAGGTTAGTTTTATTTT

Blasting.

In [45]:
conda run --name blast /usr/bin/time -v blastn -db kmer-probe/sul3.fasta -query SRR1952908.kat.3.contigs.rc.single.fasta

BLASTN 2.9.0+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: kmer-probe/sul3.fasta
           1 sequences; 792 total letters



Query= >Contig_1_52.4456 Reversed:

Length=1344
                                                                      Score        E
Sequences producing significant alignments:                          (Bits)     Value

gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli]           1463       0.0  


>gb|FJ196385|-|8531-9323|ARO:3000413|sul3 [Escherichia coli] 
Length=792

 Score = 1463 bits (792),  Expect = 0.0
 Identities = 792/792 (100%), Gaps = 0/792 (0%)
 Strand=Plus/Plus

Query  126  ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTT  185
            ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Sbjct  1    ATGAGCAAGATTTTTGGAATCGTAAATATAACCACCGATAGTTTTTCCGATGGAGGACTT  60

Query  186  TATTTAGATAC